<a href="https://colab.research.google.com/github/Amisha1019/Gen-AI-Customer-Service-Bot-Internship-Task/blob/main/Task_6_of_Gen_AI_intenrship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!streamlit_multilingual_chatbot.py

!Multilingual Customer Service Chatbot (Streamlit single-file app)

/bin/bash: line 1: streamlit_multilingual_chatbot.py: command not found
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `Multilingual Customer Service Chatbot (Streamlit single-file app)'


In [2]:
!pip install streamlit langdetect openai transformers sentence-transformers spacy faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=11ada049b6943d5b3dd0549ee18f05be78ba72bbed15d22c0e9487caddb188e7
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
!streamlit run streamlit_multilingual_chatbot.py --server.port 8501


Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: streamlit_multilingual_chatbot.py


In [5]:
import os
from typing import Tuple, Dict, Any, List
import streamlit as st


In [6]:
try:
    from langdetect import detect_langs, DetectorFactory
    DetectorFactory.seed = 0
except Exception:
    st.warning("Please install langdetect: pip install langdetect")
    raise

In [7]:
HAS_OPENAI = False
try:
    import openai
    HAS_OPENAI = True
except Exception:
    pass

In [8]:
HAS_TRANSFORMERS = False
try:
    from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
    HAS_TRANSFORMERS = True
except Exception:
    pass

In [9]:
HAS_SENTENCE_TRANSFORMERS = False
try:
    from sentence_transformers import SentenceTransformer, util
    HAS_SENTENCE_TRANSFORMERS = True
except Exception:
    pass


In [10]:
HAS_SPACY = False
try:
    import spacy
    HAS_SPACY = True
except Exception:
    pass

In [11]:
SUPPORTED_LANGUAGES = {
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "hi": "Hindi",
    "zh-cn": "Chinese (Simplified)"
}


In [12]:
CULTURAL_TEMPLATES = {
    "en": {
        "greeting": "Hi! How can I help you today?",
        "closing": "Thanks for reaching out — is there anything else?",
        "tone": "friendly, concise"
    },
    "es": {
        "greeting": "¡Hola! ¿En qué puedo ayudarte hoy?",
        "closing": "Gracias por contactarnos — ¿algo más en lo que pueda ayudar?",
        "tone": "cálido y respetuoso"
    },
    "fr": {
        "greeting": "Bonjour ! Comment puis-je vous aider aujourd'hui ?",
        "closing": "Merci de nous avoir contactés — puis-je faire autre chose pour vous ?",
        "tone": "poli et professionnel"
    },
    "hi": {
        "greeting": "नमस्ते! मैं आपकी कैसे मदद कर सकता/सकती हूँ?",
        "closing": "संपर्क करने के लिए धन्यवाद — क्या और कुछ चाहिए?",
        "tone": "सम्मानजनक और मिलनसार"
    },
    "zh-cn": {
        "greeting": "您好！ 我能为您做些什么？",
        "closing": "感谢您的联系——还有其他需要帮助的吗？",
        "tone": "礼貌和简洁"
    }
}

In [13]:
def detect_language(text: str) -> Tuple[str, float]:
    """Detect language and return (lang_code, confidence). Uses langdetect."""
    try:
        langs = detect_langs(text)
        if not langs:
            return "en", 0.0
        top = langs[0]
        code = top.lang
        # Normalize some codes
        if code == "zh-cn" or code.startswith("zh"):
            code = "zh-cn"
        if code not in SUPPORTED_LANGUAGES:
            # fallback mapping for common codes
            mapping = {"zh": "zh-cn", "hi": "hi", "en": "en", "es": "es", "fr": "fr"}
            code = mapping.get(code, "en")
        return code, float(top.prob)
    except Exception:
        return "en", 0.0

In [14]:
def get_cultural_template(lang: str) -> Dict[str, str]:
    return CULTURAL_TEMPLATES.get(lang, CULTURAL_TEMPLATES["en"])

In [15]:
INTENT_KEYWORDS = {
    "greeting": ["hello", "hi", "hey", "namaste", "hola", "bonjour"],
    "complaint": ["problem", "complaint", "not working", "broken", "refund", "issue", "delay"],
    "order_status": ["order", "tracking", "status", "delivery", "delivered", "shipped"],
    "product_info": ["price", "spec", "details", "feature", "size", "color"]
}

In [16]:
def detect_intent(text: str) -> Tuple[str, float]:
    """Very simple intent detection by keyword matching. Returns (intent, score)."""
    text_l = text.lower()
    scores = {}
    for intent, kws in INTENT_KEYWORDS.items():
        s = sum(1 for kw in kws if kw in text_l)
        scores[intent] = s
    # Pick best
    best_intent = max(scores, key=lambda k: scores[k])
    score = float(scores[best_intent]) / (len(INTENT_KEYWORDS[best_intent]) + 0.0001)
    if score == 0:
        return "unknown", 0.0
    return best_intent, score

In [19]:
def named_entities(text: str, lang: str="en") -> List[Tuple[str, str]]:
    """Return list of (entity_text, label). Only if spaCy models are installed."""
    if not HAS_SPACY:
        return []

    model_map = {
        "en": "en_core_web_sm",
        "fr": "fr_core_news_sm",
        "es": "es_core_news_sm",
        "hi": None,
        "zh-cn": "zh_core_web_sm"
    }
    model_name = model_map.get(lang)
    if model_name is None:
        return []
    try:
        nlp = spacy.load(model_name)
    except Exception:
        try:
            # Attempt to download might be needed; skip if not present
            return []
        except Exception:
            return []
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [20]:
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
_embedding_model = None
_kb_embeddings = None
_kb_texts = None

In [21]:
def init_embeddings(kb_texts: List[str]):
    global _embedding_model, _kb_embeddings, _kb_texts
    if not HAS_SENTENCE_TRANSFORMERS:
        return
    if _embedding_model is None:
        _embedding_model = SentenceTransformer(EMBEDDING_MODEL)
    _kb_texts = kb_texts
    _kb_embeddings = _embedding_model.encode(kb_texts, convert_to_tensor=True)

In [22]:
def retrieval_candidates(query: str, top_k: int = 3) -> List[Tuple[str, float]]:
    """Return top-k KB sentences + score. KB must be initialized with init_embeddings."""
    if not HAS_SENTENCE_TRANSFORMERS or _kb_embeddings is None:
        return []
    q_emb = _embedding_model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(q_emb, _kb_embeddings, top_k=top_k)[0]
    results = []
    for h in hits:
        idx = h['corpus_id']
        score = float(h['score'])
        results.append((_kb_texts[idx], score))
    return results

In [23]:
HF_TRANSLATOR = None

def init_hf_models():
    """Initialize minimal HuggingFace models if available. Lazy load to reduce startup cost."""
    global HF_TRANSLATOR, HAS_TRANSFORMERS
    if not HAS_TRANSFORMERS:
        return

    try:
        model_name = "facebook/mbart-large-50-many-to-many-mmt"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        HF_TRANSLATOR = {"model": model, "tokenizer": tokenizer, "name": model_name}
    except Exception:
        HF_TRANSLATOR = None

In [24]:
def hf_translate(text: str, source_lang: str, target_lang: str) -> str:
    """Use mBART many-to-many to translate text (fallback)."""
    if HF_TRANSLATOR is None:
        return text
    tok = HF_TRANSLATOR["tokenizer"]
    model = HF_TRANSLATOR["model"]

    code_map = {"en": "en_XX", "fr": "fr_XX", "es": "es_XX", "hi": "hi_IN", "zh-cn":"zh_CN"}
    tgt = code_map.get(target_lang, "en_XX")
    try:
        tok.src_lang = code_map.get(source_lang, "en_XX")
        encoded = tok(text, return_tensors="pt")
        generated = model.generate(**encoded, forced_bos_token_id=tok.lang_code_to_id[tgt], max_length=512)
        out = tok.batch_decode(generated, skip_special_tokens=True)[0]
        return out
    except Exception:
        return text

In [31]:
def generate_response_openai(system_prompt: str, user_message: str, lang: str, temperature: float=0.2) -> str:
    """Generate a reply using OpenAI chat completion (if key is available)."""
    if not HAS_OPENAI or "OPENAI_API_KEY" not in os.environ:
        return ""
    openai.api_key = os.environ.get("OPENAI_API_KEY")
    template = get_cultural_template(lang)
    system = f"{system_prompt}\nTone: {template.get('tone','friendly')}\nLanguage: {SUPPORTED_LANGUAGES.get(lang,'English')}"
    try:
        resp = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role":"system", "content": system},
                {"role":"user", "content": user_message}
            ],
            temperature=temperature,
            max_tokens=400
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        try:
            resp = openai.Completion.create(
                model="text-davinci-003",
                prompt=f"{system}\n\nUser: {user_message}\nAssistant:",
                temperature=temperature,
                max_tokens=400
            )
            return resp.choices[0].text.strip()
        except Exception:
            return f"(OpenAI call failed: {e})"

In [32]:
def generate_response_hf(prompt: str, lang: str, max_length=200) -> str:
    """Fallback: Use HF mBART to 'generate' a reply by translating to English, generating simple template reply, then translating back.
       This is a basic approach to avoid loading a separate generator model."""
    tpl = get_cultural_template(lang)
    reply = f"{tpl['greeting']} I understand you'd like help. {tpl['closing']}"
    return reply

In [33]:
st.set_page_config(page_title="Multilingual Customer Service Chatbot", layout="wide")
st.title("Multilingual Customer Service Chatbot")

with st.sidebar:
    st.header("Settings")
    openai_key = st.text_input("OpenAI API Key (optional)", type="password", help="If provided, OpenAI will be used for generation/translation.")
    if openai_key:
        os.environ["OPENAI_API_KEY"] = openai_key
    use_hf = st.checkbox("Prefer local HuggingFace models if available", value=False)
    if use_hf and not HAS_TRANSFORMERS:
        st.warning("Transformers not installed; local models unavailable.")
    st.markdown("**Supported languages:** " + ", ".join([f"{k} ({v})" for k,v in SUPPORTED_LANGUAGES.items()]))
    st.markdown("---")
    st.markdown("Advanced options")
    temperature = st.slider("Response creativity (temperature)", 0.0, 1.0, 0.2)
    show_ner = st.checkbox("Show Named Entities (spaCy)", value=False)
    show_retrieval = st.checkbox("Use KB retrieval fallback", value=False)


2025-11-05 10:33:15.717 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:33:15.721 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:33:15.909 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-05 10:33:15.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:33:15.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:33:15.912 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:33:15.913 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [34]:
KB_SNIPPETS = [
    "You can track your order by visiting the Orders page and entering your tracking number.",
    "We accept returns within 30 days of delivery. Items must be unused and in original packaging.",
    "Standard shipping takes 3-5 business days; express shipping takes 1-2 business days.",
    "For refunds, it can take 5-10 business days after we receive the returned item to process."
]
if show_retrieval and HAS_SENTENCE_TRANSFORMERS:
    init_embeddings(KB_SNIPPETS)


In [35]:
if show_retrieval and HAS_SENTENCE_TRANSFORMERS:
    init_embeddings(KB_SNIPPETS)

# Conversation state (Streamlit session_state)
if "history" not in st.session_state:
    st.session_state.history = []

st.subheader("Chat")

col1, col2 = st.columns([3,1])

2025-11-05 10:34:16.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:34:16.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [36]:
with col1:
    user_input = st.text_area("Your message", height=120, placeholder="Type your message here...")
    if st.button("Send"):
        if not user_input.strip():
            st.warning("Please type a message.")
        else:
            lang, conf = detect_language(user_input)
            st.session_state.history.append({"role":"user", "text": user_input, "lang": lang, "lang_conf": conf})

            intent, intent_score = detect_intent(user_input)
            ents = named_entities(user_input, lang) if show_ner else []

            retrieval_results = []
            if show_retrieval and HAS_SENTENCE_TRANSFORMERS:
                retrieval_results = retrieval_candidates(user_input, top_k=2)

            system_prompt = "You are a helpful multilingual customer service assistant for Acme Corp. Provide short, polite answers and ask clarifying questions only when needed."

            if retrieval_results and retrieval_results[0][1] > 0.4:
                system_prompt += "\nRelevant information:\n" + "\n".join([f"- {r[0]}" for r in retrieval_results])

            reply = ""
            used_provider = "none"
            if openai_key and HAS_OPENAI and not use_hf:
                used_provider = "OpenAI"
                reply = generate_response_openai(system_prompt, user_input, lang, temperature=temperature)
            else:

                used_provider = "HuggingFace (fallback)" if HAS_TRANSFORMERS else "template"
                if HAS_TRANSFORMERS:
                    if HF_TRANSLATOR is None:
                        try:
                            init_hf_models()
                        except Exception:
                            pass

                    reply = generate_response_hf(user_input, lang, max_length=200)
                else:
                    reply = generate_response_hf(user_input, lang)

            st.session_state.history.append({"role":"assistant", "text": reply, "lang": lang, "provider": used_provider})

            st.success(f"Detected language: {SUPPORTED_LANGUAGES.get(lang,'English')} (confidence {conf:.2f}) — intent: {intent} ({intent_score:.2f}) — provider: {used_provider}")
            if ents:
                st.info("Named Entities detected: " + ", ".join([f"{t}({l})" for t,l in ents]))
            if retrieval_results:
                st.info("Retrieval matches: " + "; ".join([f"{r[0][:80]}... ({r[1]:.2f})" for r in retrieval_results]))


2025-11-05 10:36:07.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.068 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:07.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [37]:
with col2:
    st.markdown("**Conversation**")
    for msg in reversed(st.session_state.history[-10:]):
        if msg['role'] == 'assistant':
            st.markdown(f"**Assistant ({msg.get('lang','en')}) — {msg.get('provider','')}**\n> {msg['text']}")
        else:
            st.markdown(f"**User ({msg.get('lang','en')})**\n> {msg['text']}")

st.markdown("---")
st.markdown("### Developer notes & extension ideas")
st.markdown("""
- To improve intent detection replace the rule-based NLU with a small classifier (fine-tune a multilingual model).
- For production, prefer OpenAI or a hosted LLM for generation; HF mT5/mBART variants can be used but are large.
- Add culturally aware templates per-country (e.g., formal vs informal pronouns in Spanish, honorifics in Japanese/Korean).
- Add logging & analytics to measure language distribution, fallback rates, and user satisfaction per language.
""")

2025-11-05 10:36:30.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.716 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 10:36:30.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()